In [1]:
import os
import glob

In [2]:
os.chdir('/xdisk/tfaily/mig2020/extra/nathaliagg/sulfate_experiment/metagenome/')

In [16]:
r1_files = glob.glob('data/data_fastq/R1/*.gz')
r2_files = glob.glob('data/data_fastq/R2/*.gz')

dict_files = {'R1':{}, 'R2':{}}

In [17]:
unique_samples = set()

for file in r1_files:
    sample_code = file.split('/')[-1].split('_')[0][4:]
    unique_samples.add(sample_code)
    file_name = file.split('/')[-1]
    if sample_code not in dict_files['R1']:
        dict_files['R1'][sample_code] = list()
        dict_files['R1'][sample_code].append("$DATADIR/R1/"+file_name)
    else:
        dict_files['R1'][sample_code].append("$DATADIR/R1/"+file_name)
    
for sample_code in dict_files['R1']:
    dict_files['R1'][sample_code] = sorted(dict_files['R1'][sample_code])
    
for file in r2_files:
    sample_code = file.split('/')[-1].split('_')[0][4:]
    file_name = file.split('/')[-1]
    if sample_code not in dict_files['R2']:
        dict_files['R2'][sample_code] = list()
        dict_files['R2'][sample_code].append("$DATADIR/R2/"+file_name)
    else:
        dict_files['R2'][sample_code].append("$DATADIR/R2/"+file_name)
        
for sample_code in dict_files['R1']:
    dict_files['R2'][sample_code] = sorted(dict_files['R2'][sample_code])

In [18]:
dict_files['R1']['50B']

['$DATADIR/R1/NGG-50B_S8_L001_R1_001.fastq.gz',
 '$DATADIR/R1/NGG-50B_S8_L002_R1_001.fastq.gz',
 '$DATADIR/R1/NGG-50B_S8_L003_R1_001.fastq.gz',
 '$DATADIR/R1/NGG-50B_S8_L004_R1_001.fastq.gz']

In [19]:
dict_files['R2']['50B']

['$DATADIR/R2/NGG-50B_S8_L001_R2_001.fastq.gz',
 '$DATADIR/R2/NGG-50B_S8_L002_R2_001.fastq.gz',
 '$DATADIR/R2/NGG-50B_S8_L003_R2_001.fastq.gz',
 '$DATADIR/R2/NGG-50B_S8_L004_R2_001.fastq.gz']

In [25]:
%%bash
SCRIPT_NAME="template"
/home/u30/nathaliagg/ngg_scripts/make_slurm_script.py -s $SCRIPT_NAME -n 1 \
                                                      -c 96 -j megahit \
                                                      -w 64:00:00 -e nathaliagg@email.arizona.edu \
                                                      -pi tfaily 
                                                      
echo "module load anaconda/2020.02
" >> $SCRIPT_NAME.slurm

echo "source activate megahit_env
" >> $SCRIPT_NAME.slurm

In [26]:
outputdir = 'megahit_output'

if not os.path.exists('megahit_output'):
    os.makedirs('megahit_output')

In [27]:
script_name="template"

for sample in sorted(unique_samples):
    with open(script_name+'.slurm', 'r') as file:
        with open('megahit_'+sample+'.slurm', 'a') as out:
            for line in file:
                out.write(line)
            
            out.write('export DATADIR="{}"\n'.format("data/data_fastq/"))
            out.write('export R1s="{}"\n'.format(",".join(dict_files['R1'][sample])))
            out.write('export R2s="{}"\n'.format(",".join(dict_files['R2'][sample])))
            
            out.write('megahit -1 $R1s -2 $R2s -m 0.90 -o megahit_output/{}_output'.format(sample))

In [28]:
%%bash
for file in megahit*slurm; do echo $file; sbatch $file; done

megahit_18A.slurm
Submitted batch job 336196
megahit_18B.slurm
Submitted batch job 336197
megahit_26A.slurm
Submitted batch job 336198
megahit_26B.slurm
Submitted batch job 336199
megahit_34A.slurm
Submitted batch job 336200
megahit_34B.slurm
Submitted batch job 336201
megahit_50A.slurm
Submitted batch job 336202
megahit_50B.slurm
Submitted batch job 336203
megahit_82A.slurm
Submitted batch job 336204
megahit_82B.slurm
Submitted batch job 336205


**Job completion**

```
megahit_18A.slurm
Submitted batch job 336196 ~ 8:10h

megahit_18B.slurm
Submitted batch job 336197 ~ 7:50h

megahit_26A.slurm
Submitted batch job 336198 ~ 9:10h

megahit_26B.slurm
Submitted batch job 336199 ~ 8:50h

megahit_34A.slurm
Submitted batch job 336200 ~ 3:50h

megahit_34B.slurm
Submitted batch job 336201 ~ 9:10h

megahit_50A.slurm
Submitted batch job 336202 ~ 6:45h

megahit_50B.slurm
Submitted batch job 336203 ~ 7:40h

megahit_82A.slurm
Submitted batch job 336204 ~ 7:50h

megahit_82B.slurm
Submitted batch job 336205 ~ 7:15h
```